In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
!pip install torch torchvision transformers pandas tqdm pillow


In [ ]:
import torch
from PIL import Image
from transformers import CLIPProcessor, CLIPModel
import pandas as pd, os, json, tqdm

subset_dir = "/content/drive/MyDrive/VPM/subset_550"
metadata_file = "/content/drive/MyDrive/VPM/subset_550_metadata.csv"

df = pd.read_csv(metadata_file)

device = "cuda" if torch.cuda.is_available() else "cpu"
model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32").to(device)
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

embeddings = {}
metadata = {}

for _, row in tqdm.tqdm(df.iterrows(), total=len(df)):
    img_path = os.path.join(subset_dir, f"{row['id']}.jpg")
    if not os.path.exists(img_path):
        continue

    image = Image.open(img_path).convert("RGB")
    inputs = processor(images=image, return_tensors="pt").to(device)

    with torch.no_grad():
        emb = model.get_image_features(**inputs)[0].cpu().tolist()

    embeddings[row['id']] = emb
    metadata[row['id']] = {
        "name": row['id'],
        "category": row['masterCategory'],
        "image": img_path.replace("\\", "/")
    }

with open("/content/drive/MyDrive/VPM/embeddings_550.json", "w") as f:
    json.dump(embeddings, f)
with open("/content/drive/MyDrive/VPM/metadata_550.json", "w") as f:
    json.dump(metadata, f)

print("Embeddings for 550 images saved in Drive!")


100%|██████████| 550/550 [01:56<00:00,  4.72it/s]


✅ Embeddings for 500 images saved in Drive!
